In [125]:
os.getcwd().split("\\")[-1]

'git_finm-portfolio-2024'

In [222]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.formula.api as smf

if os.getcwd().split("\\")[-1] == "midterm_practice":
    os.chdir("../../")

# current_dir = os.getcwd()
# if current_dir[-16:] == "midterm_practice":
#     parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
#     parent_dir = os.path.abspath(os.path.join(parent_dir, os.pardir))
    
#     os.chdir(parent_dir)
#     sys.path.insert(0, parent_dir)
import cmds.portfolio_management_helper as pmh

# Midterm 1

## FINM 36700 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

## Please note the following:

Points
* The exam is 100 points.
* You have 120 minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.
Final Exam

Submission
* You will upload your solution to the `Midterm 1` assignment on Canvas, where you downloaded this. (Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers,) and it should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

# Instructions

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_data_1.xlsx`

This file has sheets for...
* `info` - names of each stock ticker
* `excess returns` - weekly excess returns on several stocks
* `SPY` - weekly excess returns on SPY

Note the data is **weekly** so any annualizations should use `52` weeks in a year.

#### If useful
here is code to load in the data.

In [127]:
file_path = r"review/midterm_practice/midterm_1_data.xlsx"
dfs_raw = pd.read_excel(file_path, sheet_name=None)
for key in dfs_raw.keys():
    print(f"{key}: {dfs_raw[key].shape}")

ticker_mapping = {tick: name 
                  for tick, name in zip(dfs_raw['info'].iloc[:, 0], 
                                                dfs_raw['info'].iloc[:, 1])}
ticker_mapping

info: (7, 2)
excess returns: (392, 8)
spy: (392, 2)


{'AAPL': 'Apple Inc',
 'MSFT': 'Microsoft Corp',
 'AMZN': 'Amazon.com Inc',
 'NVDA': 'NVIDIA Corp',
 'GOOGL': 'Alphabet Inc',
 'TSLA': 'Tesla Inc',
 'XOM': 'Exxon Mobil Corp'}

In [128]:
df_excess_rets = dfs_raw['excess returns'].set_index("date")
df_spy = dfs_raw['spy'].set_index("date")

display(df_excess_rets.head())
display(df_spy.head())

annual_factor = 52

,AAPL,MSFT,AMZN,NVDA,GOOGL,TSLA,XOM
date,,,,,,,
2016-01-15,-0.0061,-0.0334,-0.0686,-0.0929,-0.0358,-0.0363,0.0309
2016-01-22,0.0453,0.0266,0.0471,0.0505,0.0503,-0.0108,-0.0119
2016-01-29,-0.0517,0.0421,-0.0272,0.0180,0.0098,-0.0675,0.0052
2016-02-05,-0.0362,-0.0968,-0.1519,-0.1050,-0.0830,-0.1569,0.0213
2016-02-12,-0.0079,-0.0008,0.0023,-0.0341,-0.0031,-0.0787,0.0135


,SPY
date,
2016-01-15,-0.0214
2016-01-22,0.0144
2016-01-29,0.0168
2016-02-05,-0.0298
2016-02-12,-0.0070


In [129]:
# FILEIN = 'midterm_1_data.xlsx'
# sheet_exrets = 'excess returns'
# sheet_spy = 'spy'

# retsx = pd.read_excel(FILEIN, sheet_name=sheet_exrets).set_index('date')
# spy = pd.read_excel(FILEIN, sheet_name=sheet_spy).set_index('date')

# print(f"retsx: {retsx.shape}")
# print(f"spy: {spy.shape}")

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 20     |
| 2       | 35     |
| 3       | 30     |
| 4       | 15     |

### Each numbered question is worth 5 points.

### Notation
(Hidden LaTeX commands)

$$\newcommand{\mux}{\tilde{\boldsymbol{\mu}}}$$
$$\newcommand{\wtan}{\boldsymbol{\text{w}}^{\text{tan}}}$$
$$\newcommand{\wtarg}{\boldsymbol{\text{w}}^{\text{port}}}$$
$$\newcommand{\mutarg}{\tilde{\boldsymbol{\mu}}^{\text{port}}}$$
$$\newcommand{\wEW}{\boldsymbol{\text{w}}^{\text{EW}}}$$
$$\newcommand{\wRP}{\boldsymbol{\text{w}}^{\text{RP}}}$$
$$\newcommand{\wREG}{\boldsymbol{\text{w}}^{\text{REG}}}$$

# 1. Short Answer

### No Data Needed

These problem does not require any data file. Rather, analyze the situation conceptually, based on the information below. 

## 1

In what sense was ProShares `HDG` successful in hedging the `HFRI`, and in what sense was it unsuccessful in tracking the `HFRI`?

It had high correlation to HFRI. It captures the most notable properties of HFRI. Similar risk exposure

Less returns then HFRI.

## 2

We discussed multiple ways of calculating Value-at-Risk (VaR). What are the tradeoffs between using the normal distribution formula versus a directly empirical approach?

Typically returns are fat tailed. So using a normal distribution will make the cVaR very unlikely

## 3

Did we find that **TIPS** have been useful in expanding the mean-variance frontier in the past? Did we conclude they might be useful in the future? Explain.

We should include it because it has low correlation with other assets

## 4.

What aspect of the classic mean-variance optimization approach leads to extreme answers? How did regularization help with this issue?

When one asset's is equal (or can be approximated by) to a linear combination of the other asset returns.

***

# 2. Allocation

Consider a mean-variance optimization of **excess** returns provided in `midterm_1_data.xlsx.`

## 1. 

Report the following **annualized** statistics:
* mean
* volatility
* Sharpe ratio

Which assets have the highest / lowest Sharpe ratios?

In [130]:
def calc_summary(df, annual_factor=1):
    return df.apply(lambda x: pd.Series({
        "mean": x.mean() * annual_factor,
        "std": x.std() * np.sqrt(annual_factor),
        "sharpe": x.mean() / x.std() * np.sqrt(annual_factor)  
    }))

df_summary = (calc_summary(df_excess_rets, annual_factor=annual_factor)
              # .rename(columns=custom_ticker_mapping)
              ).T

display(df_summary)

,mean,std,sharpe
AAPL,0.3194,0.2839,1.1252
MSFT,0.2881,0.2402,1.1993
AMZN,0.2395,0.3104,0.7715
NVDA,0.6507,0.4681,1.3900
GOOGL,0.1933,0.2742,0.7050
TSLA,0.5697,0.6070,0.9386
XOM,0.1242,0.3116,0.3986


## 2.

Report the weights of the tangency portfolio.

Also report the Sharpe ratio achieved by the tangency portfolio over this sample.

In [131]:
def compute_stats(weights, mu_hat, sigma_hat, annual_factor=12):
    p_return = weights.T @ mu_hat * annual_factor
    p_vol = np.sqrt(weights.T @ sigma_hat @ weights) * np.sqrt(annual_factor)
    return p_vol, p_return

def tangent_portfolio(mu_hat, sigma_hat, flip_weights=False, annual_factor=12):
    sigma_inv = np.linalg.inv(sigma_hat)
    weights = sigma_inv @ mu_hat
    weights = weights / np.sum(weights)
    
    t_vol, t_return = compute_stats(weights, mu_hat, sigma_hat, annual_factor=annual_factor)
    if t_return < 0 and flip_weights:
        weights *= -1
        t_vol, t_return = compute_stats(weights, mu_hat, sigma_hat, annual_factor=annual_factor)
    return [t_vol, t_return, weights]

mu_hat = df_excess_rets.mean().values
sigma_hat = df_excess_rets.cov().values
t_vol, t_return, t_weights = tangent_portfolio(mu_hat, sigma_hat, annual_factor=annual_factor)

print(f"tangent mean: {t_return}")
print(f"tangent vol: {t_vol }")
print(f"tangent sharpe: {t_return/t_vol }")

df_temp = pd.DataFrame(t_weights, index=df_excess_rets.columns, columns=["tangent_weight"])
df_temp = pd.merge(df_temp, df_summary,left_index=True, right_index=True)
display(df_temp.sort_values("sharpe", ascending=False))


tangent mean: 0.5634741093289569
tangent vol: 0.35835079239709444
tangent sharpe: 1.5724092740516726


,tangent_weight,mean,std,sharpe
NVDA,0.4960,0.6507,0.4681,1.3900
MSFT,0.7875,0.2881,0.2402,1.1993
AAPL,0.3226,0.3194,0.2839,1.1252
TSLA,0.1060,0.5697,0.6070,0.9386
AMZN,-0.2286,0.2395,0.3104,0.7715
GOOGL,-0.5027,0.1933,0.2742,0.7050
XOM,0.0193,0.1242,0.3116,0.3986


In [132]:
# analysis_sharpe_vs_tangent_weights = (
#     pmh.calc_summary_statistics(
#         df_excess_rets,
#         annual_factor=annual_factor,
#         provided_excess_returns=True,
#         keep_columns=['Tangency Weights', 'Annualized Sharpe']
#     )
#     .sort_values('Annualized Sharpe', ascending=False)
# )

# display(analysis_sharpe_vs_tangent_weights)

In [133]:
# pmh.calc_summary_statistics(
#     pmh.calc_tangency_weights(df_excess_rets, return_port_ret=True),
#     annual_factor=annual_factor,
#     provided_excess_returns=True,
#     keep_columns=['Annualized Sharpe', 'Annualized Mean', 'Annualized Vol']
# )

## 3.

* What weight is given to the asset with the lowest Sharpe ratio?
* What Sharpe ratio does the lowest (most negative) weight asset have?

Explain. Support your answer with evidence.

XOM has the lowest sharpe ratio and this was given a weight of .0193. GOOGL has the lowest weight with a sharpe ratio of .7050

## 4.

Let's examine the out-of-sample performance.

Calculate and report the following three allocations using only data through the end of 2022:
* tangency portfolio
* equally weighted portfolio
* a regularized approach, with a new formula shown below

where
$$\wEW_i = \frac{1}{n}$$

$$\wREG \sim \widehat{\Sigma}^{-1}\mux$$

$$\widehat{\Sigma} = \frac{\Sigma + \boldsymbol{2}\,\Sigma_D}{\boldsymbol{3}}$$
where $\Sigma_D$ denotes a *diagonal* matrix of the security variances, with zeros in the off-diagonals.

In [150]:
def create_portfolios(df_returns):
    mu_hat=df_returns.mean().values
    sigma_hat=df_returns.cov().values
    
    port_weights = {
        'Tangent': tangent_portfolio(mu_hat, sigma_hat, annual_factor=annual_factor)[2],
        'Equally-weighting': np.ones_like(mu_hat)/len(mu_hat),
        'Regularized': tangent_portfolio(mu_hat, 
                                         (sigma_hat + 2*np.diag(np.diagonal(sigma_hat)) ) / 3
                                         )[2],
    }
    
    # Returns
    df_port_returns = pd.DataFrame({name : df_returns @ weights 
                                         for name, weights in port_weights.items()})
    
    # Statistics
    df_port_stats = calc_summary(df_port_returns, annual_factor=annual_factor)
    for name, weights in port_weights.items():
        df_port_stats.loc["sum weights", name] = sum(weights)
    return port_weights, df_port_returns, df_port_stats


df_2022 = df_excess_rets[df_excess_rets.index.year <= 2022]

port_weights, df_port_returns, df_port_stats = create_portfolios(df_2022)

# Check target
for name, weights in port_weights.items():
    print(f"{name}: {weights.dot(df_excess_rets.mean()*annual_factor):.6f}")

Tangent: 0.524255
Equally-weighting: 0.340696
Regularized: 0.374730


In [143]:
df_port_weights = pd.DataFrame(port_weights, index=df_excess_rets.columns)

display(df_port_weights)

,Tangent,Equally-weighting,Regularized
AAPL,0.3106,0.1429,0.2373
MSFT,1.0731,0.1429,0.3308
AMZN,-0.2591,0.1429,0.0472
NVDA,0.3801,0.1429,0.1968
GOOGL,-0.7515,0.1429,0.0114
TSLA,0.1016,0.1429,0.0902
XOM,0.1453,0.1429,0.0864


## 5.

Report the out-of-sample (2023) performance of all three portfolios in terms of annualized mean, vol, and Sharpe.

In [144]:
df_2023 = df_excess_rets[df_excess_rets.index.year == 2023]

calc_summary(df_2023 @ df_port_weights,  annual_factor=annual_factor)

,Tangent,Equally-weighting,Regularized
mean,1.2047,0.9551,1.0135
std,0.4437,0.2470,0.2503
sharpe,2.7150,3.8677,4.0499


## 6.

Imagine just for this problem that this data is for **total** returns, not excess returns.

Report the weights of the global-minimum-variance portfolio.

In [185]:
def gmv_weights(tot_returns):
    
    ones = np.ones(tot_returns.columns.shape)
    cov = tot_returns.cov()*12
    cov_inv = np.linalg.inv(cov)
    scaling = 1/(np.transpose(ones) @ cov_inv @ ones)
    gmv_tot = scaling * cov_inv @ ones
    return gmv_tot

df_port_weights["GMV"] = gmv_weights(df_excess_rets)

display(df_port_weights)
calc_summary(df_2023 @ df_port_weights, annual_factor)

,Tangent,Equally-weighting,Regularized,GMV
AAPL,0.3106,0.1429,0.2373,0.2062
MSFT,1.0731,0.1429,0.3308,0.4912
AMZN,-0.2591,0.1429,0.0472,0.1609
NVDA,0.3801,0.1429,0.1968,-0.1192
GOOGL,-0.7515,0.1429,0.0114,0.0114
TSLA,0.1016,0.1429,0.0902,-0.0469
XOM,0.1453,0.1429,0.0864,0.2964


,Tangent,Equally-weighting,Regularized,GMV
mean,1.2047,0.9551,1.0135,0.2431
std,0.4437,0.2470,0.2503,0.1835
sharpe,2.7150,3.8677,4.0499,1.3249


## 7.

To target a mean return of 0.005%, would you be long or short this global minimum variance portfolio?

In [141]:
annual_factor * .00005

0.0026000000000000003

long the global minimum

***

# 3. Performance

## 1. 

Report the following performance metrics of excess returns for Tesla (`TSLA`).
* skewness
* kurtosis

You are not annualizing any of these stats.

What do these metrics indicate about the nature of the returns?

In [203]:
tsla_rets = df_excess_rets["TSLA"]


print(f"Skewness: {tsla_rets.skew():.3f}")
print(f"Excess Kurtosis: {tsla_rets.kurtosis():.3f}")

Skewness: 0.441
Excess Kurtosis: 1.527


The distribution is positively (right skewed) meaning the median return will be larger then the median.

The kurtosis is 1.527 which is less then 3. This tells us the distribution has fat tails. So crazy returns are more likely to be seen then if it was assumed to be normal

## 2. 

Report the maximum drawdown for `TSLA` over the sample.
* Ignore that your data is in excess returns rather than total returns.
* Simply proceed with the excess return data for this calculation.

In [206]:
tsla_cum_rets = (tsla_rets+1).cumprod()
draw_downs = (tsla_cum_rets.cummax() - tsla_cum_rets) / tsla_cum_rets.cummax()

print(f"Max Draw Down: {draw_downs.max():.3f}")

Max Draw Down: 0.682


## 3.

For `TSLA`, calculate the following metrics, relative to `SPY`:
* market beta
* alpha
* sortino ratio

Annualize alpha and sortino ratio.

In [220]:
def factor_model(df, y_var, x_vars, intercept=True):
    if not isinstance(x_vars, list):
        x_vars = [x_vars]
    # Run regression
    formula = f"{y_var} ~ {' + '.join(x_vars)}"
    if not intercept:
        formula = formula + " - 1"
    model = smf.ols(formula=formula, data=df)
    results = model.fit()
    summary = results.summary()
    
    return model, results

def risk_summary(df, index_cols, market_col, annual_scale = 12):
    
    data = []
    for c in index_cols:
        _, results = factor_model(df, c, market_col)
        market_beta = results.params[market_col]
        
        data.append({
            "index": c,
            "Market Alpha": results.params["Intercept"] * annual_scale,
            "Market Beta": market_beta,
            "Sortino": df[c].mean() / df[c][df[c] < 0].std() * np.sqrt(annual_scale),
        })
    return pd.DataFrame(data).set_index("index")

df_regression = pd.DataFrame()
df_regression["TSLA"] = tsla_rets
df_regression["SPY"] = df_spy["SPY"]
# display(df_regression)

risk_summary(df_regression, ["TSLA"], "SPY", annual_scale=annual_factor)

,Market Alpha,Market Beta,Sortino
index,,,
TSLA,0.3095,1.7768,1.6423


## 4.

Continuing with `TSLA`, calculate the full-sample, 5th-percentile CVaR.
* Use the `normal` formula, assuming mean returns are zero.
* Use the full-sample volatility.

Use the entire sample to calculate a single CVaR number. 

In [226]:
percent_cvar = 0.05
cvar_multiplier = -1 * stats.norm.pdf(stats.norm.ppf(percent_cvar)) / percent_cvar

print(f"cVaR: {tsla_rets.std() * cvar_multiplier}")


cVaR: -0.1736378370038865


## 5.

Now calculate the 5th-percentile, one-period ahead, **VaR** for `TSLA`.

Here, calculate the running series of VaR estimates.

Again, 
* use the normal formula, with mean zero.

But now, use the rolling volatility, based on 
* rolling window or $m=52$ weeks.

Report the final 5 values of your calculated VaR series.

In [258]:
var_multiplier = stats.norm.ppf(percent_cvar)
# var_multiplier = -1.65

vars = (
    tsla_rets
    .rolling(window=annual_factor)
    .apply(lambda x: np.sqrt(sum(x**2)/len(x)))
    .shift(1) 
    * var_multiplier
)

vars.tail(5)

date
2023-06-16   -0.1561
2023-06-23   -0.1563
2023-06-30   -0.1537
2023-07-07   -0.1522
2023-07-14   -0.1505
Name: TSLA, dtype: float64

## 6. 

Calculate the out-of-sample **hit ratio** for your VaR series reported in your previous answer.

In [260]:
def get_hit_percent(vars, returns):
    return sum(returns < vars) / sum(~vars.isna())

exceed = get_hit_percent(vars, tsla_rets)
print(f'Exceed Percent - 5% VaR Expanding: {exceed:.2%}')

Exceed Percent - 5% VaR Expanding: 5.59%


***

# 4. Hedging

## 1. 

Consider the following scenario: you are holding a \$100 million long position in `NVDA`. You wish to hedge the position using some combination of 
* `AAPL`
* `AMZN`
* `GOOGL`
* `MSFT`

Report the positions you would hold of those 4 securities for an optimal hedge.

Note:
* In the regression estimation, include an intercept.
* Use the full-sample regression. No need to worry about in-sample versus out-of-sample.

## 2.

How well does the hedge do? Cite a regression statistic to support your answer.

Also estimate the volatility of the basis, (epsilon.)

## 3.

Report the annualized intercept. By including this intercept, what are you assuming about the nature of the returns of `NVDA` as well as the returns of the hedging instruments?